In [1]:
#import tensorflow as tf
#!pip install tensorflow==2.9.1
import tensorflow as tf
print(tf.__version__)

2.9.1


In [2]:
print(tf.__version__)
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal
#import pywt
import numpy as np
#import imageio
import tensorflow as tf
#import cv2
#import time
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy
from tensorflow.keras.models import Model, load_model
from sklearn.metrics import classification_report
import os
#from scipy import signal
#from sklearn import preprocessing
print(tf.__version__)
from google.colab import drive
drive.mount('/content/drive')

2.9.1
2.9.1
Mounted at /content/drive


In [3]:
train_data = pd.read_csv('/content/drive/MyDrive/TCC-Physionet/dadosPhysionetTrainBalanceado.csv', delimiter=";")
test_data = pd.read_csv('/content/drive/MyDrive/TCC-Physionet/dadosPhysionetTestBalanceado.csv', delimiter=";")
val_data = pd.read_csv('/content/drive/MyDrive/TCC-Physionet/dadosPhysionetValBalanceado.csv', delimiter=";")


In [4]:
labels = {'0': 'rest', '1': 'left', '2': 'right'}
selected_eletrodes = ['F3', 'F4', 'F5', 'F6', 'FC5', 'FC3',
             'FC4', 'FC6', 'C5', 'C3', 'C4', 'C6'
             ]

In [5]:
def assemble_data(data, selected_eletrodes, std_el, mean_el):

  labels = []
  signals = []
  pacients = data["paciente"].unique()
  len_selected_eletrodes = len(selected_eletrodes)

  for pacient in pacients:
      measures = data.loc[data['paciente'] == pacient]['medicao'].unique()
      for measure in measures:
          label = data.loc[(data['paciente'] == pacient) & (data['medicao'] == measure)]['label'].unique()[0]
          signal_list = []
          img = np.zeros([len_selected_eletrodes,640])
          for idx, eletrode in enumerate(selected_eletrodes):
              sign = data.loc[(data['paciente'] == pacient) & (data['medicao'] == measure) & (data['eletrodo'] == eletrode)].iloc[:,4:].to_numpy().reshape(640)
              sign -= mean_el[idx]
              sign /= std_el[idx] + 1e-7
              img[idx] = sign
          signals.append(img)
          labels.append(label-1)
          '''if label == 1 :
            labels.append([1,0,0])
          elif label == 2:
            labels.append([0,1,0])
          else:
            labels.append([0,0,1])'''

  return labels, signals


In [6]:
def metrics(data, selected_eletrodes):
  std = []
  mean = []
  for eletrode in selected_eletrodes:
    eletrodoStd = pd.Series(data.loc[data["eletrodo"] == eletrode, "leitura0":"leitura639"].values.ravel('F')).std()
    eletrodoMean = pd.Series(data.loc[data["eletrodo"] == eletrode, "leitura0":"leitura639"].values.ravel('F')).mean()
    std.append(eletrodoStd)
    mean.append(eletrodoMean)

  return std, mean

In [7]:
class PhysionetDataset:
  def __init__(self, data, selected_eletrodes, batch_size):
    self.std, self.mean = metrics(data, selected_eletrodes)
    label, signal = assemble_data(data, selected_eletrodes, self.std, self.mean)
    img_generator = tf.keras.preprocessing.image.ImageDataGenerator()

    self.features = img_generator.flow(
        x=np.array(signal).reshape(len(signal), len(selected_eletrodes), 640, 1),
        y=np.array(label).reshape(len(label),1),
        batch_size=batch_size)



In [8]:
batch = 8
cwt_limit = 30

train_dataset = PhysionetDataset(train_data, selected_eletrodes, batch)
val_dataset = PhysionetDataset(val_data, selected_eletrodes, batch)
#test_dataset = PhysionetDataset(test_data, selected_eletrodes, batch)

In [10]:
inp = layers.Input(shape=(12, 640, 1))
x = layers.TimeDistributed(layers.LSTM(256, return_sequences=True))(inp)
x = layers.TimeDistributed(layers.LSTM(128, return_sequences=True))(x)
x = layers.TimeDistributed(layers.LSTM(64, return_sequences=True))(x)
x = layers.Flatten()(x)
x = layers.Dense(3, activation='softmax')(x)

model = Model(inp, x)

In [11]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 12, 640, 1)]      0         
                                                                 
 time_distributed (TimeDistr  (None, 12, 640, 256)     264192    
 ibuted)                                                         
                                                                 
 time_distributed_1 (TimeDis  (None, 12, 640, 128)     197120    
 tributed)                                                       
                                                                 
 time_distributed_2 (TimeDis  (None, 12, 640, 64)      49408     
 tributed)                                                       
                                                                 
 flatten (Flatten)           (None, 491520)            0         
                                                             

In [12]:

model.compile(optimizer='adam',loss=tf.keras.losses.SparseCategoricalCrossentropy(),metrics=['accuracy'])

In [13]:
#model = load_model('/content/drive/MyDrive/TCC-Physionet/model_EffB309_07_2023_2.h5')

history = model.fit(train_dataset.features.x, train_dataset.features.y, epochs=10,
                    validation_data=(val_dataset.features.x, val_dataset.features.y))#, callbacks=callbacks)

#model_EffB3.save('/content/drive/MyDrive/TCC-Physionet/model_EffB309_07_2023_2.h5')

Epoch 1/10
50/50 [==============================] - 1298s 26s/step - loss: 1.3131 - accuracy: 0.3936 - val_loss: 1.0536 - val_accuracy: 0.4581
Epoch 2/10
50/50 [==============================] - 1277s 26s/step - loss: 1.0113 - accuracy: 0.5275 - val_loss: 1.1454 - val_accuracy: 0.4721
Epoch 3/10
50/50 [==============================] - 1278s 26s/step - loss: 0.9461 - accuracy: 0.5742 - val_loss: 1.0944 - val_accuracy: 0.4972
Epoch 4/10
50/50 [==============================] - 1273s 25s/step - loss: 0.9029 - accuracy: 0.5875 - val_loss: 1.0312 - val_accuracy: 0.5307
Epoch 5/10
50/50 [==============================] - 1285s 26s/step - loss: 0.8211 - accuracy: 0.6399 - val_loss: 1.1316 - val_accuracy: 0.5028
Epoch 6/10
50/50 [==============================] - 1306s 26s/step - loss: 0.8322 - accuracy: 0.6191 - val_loss: 1.2197 - val_accuracy: 0.5251
Epoch 7/10
50/50 [==============================] - 1314s 26s/step - loss: 0.7996 - accuracy: 0.6570 - val_loss: 1.0829 - val_accuracy: 0.5307

In [14]:
model.save('/content/drive/MyDrive/TCC-Physionet/model_LSTM.h5')

In [17]:
score = model.evaluate(test_dataset.features.x, test_dataset.features.y, verbose=0)
print('Loss: ', score[0])
print('Accuracy: ', score[1])

Loss:  1.4860644340515137
Accuracy:  0.4898989796638489


In [16]:
test_dataset = PhysionetDataset(test_data, selected_eletrodes, batch)